# Syncronization of Microscope frame to camera frames


Loading the Camera csv(video_timestamp) and converting into a data frame in milliseconds


In [ ]:
# Importing the packages

import pandas as pd
import numpy as np

In [61]:
# Load the data files
camera_df = pd.read_csv('camera.csv',sep=r'\s+', names= ['hrs','min','sec','mill'])
microscope_df = pd.read_csv('sync_2019-10-01T13_28_00 - Copy.csv',sep=r'\s+',names= ['hrs','min','sec','mill','frame','frame_no'])

In [62]:
camera_df.head(2)

,hrs,min,sec,mill
0,13,28,0,457
1,13,28,0,464


In [63]:
camera_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55151 entries, 0 to 55150
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   hrs     55151 non-null  int64
 1   min     55151 non-null  int64
 2   sec     55151 non-null  int64
 3   mill    55151 non-null  int64
dtypes: int64(4)
memory usage: 1.7 MB


In [64]:
microscope_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27011 entries, 0 to 27010
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   hrs       27011 non-null  int64 
 1   min       27011 non-null  int64 
 2   sec       27011 non-null  int64 
 3   mill      27011 non-null  int64 
 4   frame     27011 non-null  object
 5   frame_no  27011 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 1.2+ MB


In [65]:
microscope_df.head(2)

,hrs,min,sec,mill,frame,frame_no
0,13,28,9,867,"Fra,",2404444129
1,13,28,9,899,"Fra,",2404444163


In [66]:
# Convert time hr:min:sec:mill to milliseconds and creating a new column with millisec

def convert_to_ms(df):
   
    """ Convert hrs, min, sec, mill columns to total milliseconds and create a new column 'time_ms'. """
    return (df['hrs']*3600000 + df['min']*60000 + df['sec']*1000 + df['mill'])
   

In [67]:
camera_df['camera_time_in_ms'] = convert_to_ms(camera_df)
microscope_df['microscope_time_in_ms'] = convert_to_ms(microscope_df)

In [68]:
camera_df.head(2)

,hrs,min,sec,mill,camera_time_in_ms
0,13,28,0,457,48480457
1,13,28,0,464,48480464


In [69]:
camera_df.isnull().sum()

hrs                  0
min                  0
sec                  0
mill                 0
camera_time_in_ms    0
dtype: int64

In [70]:
microscope_df.isnull().sum()

hrs                      0
min                      0
sec                      0
mill                     0
frame                    0
frame_no                 0
microscope_time_in_ms    0
dtype: int64

In [71]:
camera_df[camera_df.duplicated(subset=['hrs','min','sec','mill','camera_time_in_ms'])]

,hrs,min,sec,mill,camera_time_in_ms


In [72]:
microscope_df[microscope_df.duplicated(subset=['hrs','min','sec','mill','frame','frame_no','microscope_time_in_ms'])]

,hrs,min,sec,mill,frame,frame_no,microscope_time_in_ms


# Observations and insights

No dupicates found in two files and no null values were found

Deleting the hr, min, sec, mill columns and keeping only millisecond column

In [73]:
camera_df.drop(columns=['hrs','min','sec','mill'], inplace =True)

In [74]:
microscope_df.drop(columns=['hrs','min','sec','mill','frame'],inplace=True)

In [75]:
microscope_df.head(3)

,frame_no,microscope_time_in_ms
0,2404444129,48489867
1,2404444163,48489899
2,2404444195,48489931


In [76]:
# Reordering the columns in micorscope_df

microscope_df = microscope_df[['microscope_time_in_ms','frame_no']]

In [77]:
microscope_df.head(3)

,microscope_time_in_ms,frame_no
0,48489867,2404444129
1,48489899,2404444163
2,48489931,2404444195


In [78]:
camera_df.head()

,camera_time_in_ms
0,48480457
1,48480464
2,48480479
3,48480496
4,48480512


In [79]:
microscope_df.tail()

,microscope_time_in_ms,frame_no
27006,49390531,2405344747
27007,49390564,2405344780
27008,49390596,2405344814
27009,49390633,2405344846
27010,49390666,2405344880


In [80]:
camera_df.tail()

,camera_time_in_ms
55146,49399783
55147,49399800
55148,49399817
55149,49399834
55150,49399850


In [81]:
# Ensuring both were sorted by time

camera_df = camera_df.sort_values('camera_time_in_ms').reset_index(drop=True)
microscope_df = microscope_df.sort_values('microscope_time_in_ms').reset_index(drop = True)

In [82]:
def synchronize_two_to_one(camera_df, microscope_df):
    """
    Synchronize 2 camera frames to 1 microscope frame.
    Camera always leads → only take camera frames before or equal to each microscope frame to reduce jetter effect
    
    """
    synced_data = []
    camera_used = set()
    expected_ratio = 2

    for i, micro_time in enumerate(microscope_df['microscope_time_in_ms']):
        # Filter camera frames before or equal to microscope timestamp
        valid_cams = camera_df[camera_df['camera_time_in_ms'] <= micro_time]

        if len(valid_cams) < expected_ratio:
            continue  # skip if not enough camera frames before microscope frame

        # Take the last 2 camera frames before microscope frame
        mapped_cams = valid_cams.tail(expected_ratio)['camera_time_in_ms'].tolist()

        synced_data.append({
            'microscope_frame_no': microscope_df.loc[i, 'frame_no'],
            'microscope_time_in_ms': micro_time,
            'camera_times': mapped_cams
        })

        # Mark used frames
        camera_used.update(valid_cams.tail(expected_ratio).index.tolist())

    synced_df = pd.DataFrame(synced_data)
    unused_cameras = camera_df.drop(index=list(camera_used)).reset_index(drop=True)

    return synced_df, unused_cameras

In [83]:
# --- Perform synchronization ---
final_synced_df, unused_camera_frames = synchronize_two_to_one(camera_df, microscope_df)


In [84]:
final_synced_df.head(20)

,microscope_frame_no,microscope_time_in_ms,camera_times
0,2404444129,48489867,"[48489848, 48489865]"
1,2404444163,48489899,"[48489881, 48489898]"
2,2404444195,48489931,"[48489915, 48489931]"
3,2404444229,48489964,"[48489948, 48489964]"
4,2404444262,48490001,"[48489981, 48489998]"
5,2404444296,48490034,"[48490014, 48490031]"
6,2404444328,48490067,"[48490048, 48490064]"
7,2404444362,48490099,"[48490081, 48490098]"
8,2404444396,48490132,"[48490115, 48490131]"
9,2404444429,48490165,"[48490148, 48490164]"


In [85]:
len(unused_camera_frames)


1925

In [86]:
# to convert the final dataframe to csv file
final_synced_df.to_csv('final_sync.csv', index=False, header=False)  


# Observations and Insights

1.Successfully synchronized the camera and microscope time frames using a 1-to-2 mapping, considering the camera frame rate is approximately double the microscope frame rate.

2.To minimize jitter effects, mapping was done in one consistent direction  (from microscope to camera frames).

3.Unused camera frames were tracked separately in a dedicated DataFrame for further analysis.